In [2]:
import numpy as np
import pandas as pd
import redis

In [3]:
r = redis.StrictRedis(host='redis-13644.c270.us-east-1-3.ec2.cloud.redislabs.com', port=13644, password='W3ajhhIRdKp8DUHt6CuTgOSQcg5FdXQ1')

In [4]:
r.ping()

True

In [5]:
r.keys()

[b'foo', b'KEY', b'ESSAGE', b'MESSAGE', b'academy:register']

In [6]:
np_file=np.load('datafrane students teacher.npz',allow_pickle=True)
x_values=np_file['arr_0']
col_names=np_file['arr_1']
df=pd.DataFrame(x_values,columns=col_names)
df

,Name,Role,Faciafeatures
0,Angelina Jolie,student,"[0.43242058, 1.0748497, -0.128624, -1.8332914,..."
1,Barack Obama,teacher,"[0.94858074, -0.5584053, 0.094820514, -1.27965..."
2,Chris Evans,student,"[0.72017145, -1.468415, -0.7781255, -1.4601074..."
3,Morgan Freeman,teacher,"[0.48535085, -0.61925495, -0.4069931, 1.068778..."
4,Scarlett Johansson,student,"[0.19131936, -0.4884374, -1.6902335, 0.4423968..."


In [7]:
df['name_role']=df['Name']+'@'+df['Role']
df

,Name,Role,Faciafeatures,name_role
0,Angelina Jolie,student,"[0.43242058, 1.0748497, -0.128624, -1.8332914,...",Angelina Jolie@student
1,Barack Obama,teacher,"[0.94858074, -0.5584053, 0.094820514, -1.27965...",Barack Obama@teacher
2,Chris Evans,student,"[0.72017145, -1.468415, -0.7781255, -1.4601074...",Chris Evans@student
3,Morgan Freeman,teacher,"[0.48535085, -0.61925495, -0.4069931, 1.068778...",Morgan Freeman@teacher
4,Scarlett Johansson,student,"[0.19131936, -0.4884374, -1.6902335, 0.4423968...",Scarlett Johansson@student


In [8]:
records=df[['name_role','Faciafeatures']].to_dict(orient='records')

In [9]:
for record in records:
    name_role=record['name_role']
    vector=record['Faciafeatures']
    
    vector_bytes=vector.tobytes()
    r.hset(name='academy:register',key=name_role,value=vector_bytes)
    

In [10]:
import os
import cv2
from insightface.app import FaceAnalysis

In [11]:
faceapp= FaceAnalysis(name='buffalo_sc',root='insightface_model',providers=['CPUExecutionProvider'])
faceapp.prepare(ctx_id=0,det_size=(640,640),det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [ ]:
person_name=input('Enter Your Name: ')

tries = 3
for i in range(tries):
    role=int(input("""
    Please Choose
    1. Student
    2. Teacher
    
    Enter number either 1 or 2
    """))
    
    if role not in (1,2):
        print('Invalid Entry')
        if i==3:
            print('Exceed maximum tries')
            
    else:
        if role==1:
            role='Student'
        else:
            role='Teacher'
        break
            
key = person_name+'@'+role
print('\nYour Name = ',person_name)
print('Your role = ',role)
print('key = ',key)

In [ ]:
cap = cv2.VideoCapture(0)
embedding_list = []
samples=0
while True:
    ret,frame=cap.read()
    if ret==False:
        print('Unable to read camera')
        break
        
    results=faceapp.get(frame,max_num=1)
    for res in results:
        samples+=1
        x1,y1,x2,y2=res['bbox'].astype(int)
        embedding=res['embedding']
        embedding_list.append(embedding)
        
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),1)
        
        
    if samples >= 100:
        break
        
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) == ord('q'):
        break
        
cv2.destroyAllWindows()
cap.release()
        
        
        
        
        
        

In [14]:
facial_features=np.asarray(embedding_list).mean(axis=0)
facial_features.shape

(512,)

In [17]:
facial_features_bytes=facial_features.tobytes()
r.hset(name='academy:register',key=key,value=facial_features_bytes)

NameError: name 'key' is not defined

In [15]:
len(embedding_list)

0